# CrewAI: Conceptos Core - Crews (Equipos)

## Introducción

Un **Crew** (equipo) en CrewAI representa un grupo colaborativo de agentes que trabajan juntos para lograr un conjunto de tareas. Cada crew define la estrategia para la ejecución de tareas, la colaboración entre agentes y el flujo de trabajo general.

### ¿Qué es un Crew?

- **Definición**: Un grupo de agentes que colaboran para completar tareas
- **Propósito**: Coordinar múltiples agentes para lograr objetivos complejos
- **Beneficios**: División de trabajo, especialización, eficiencia

### Atributos Principales de un Crew

| Atributo | Descripción | Obligatorio |
|----------|-------------|-------------|
| **Tasks** | Lista de tareas asignadas al crew | ✅ Sí |
| **Agents** | Lista de agentes que forman parte del crew | ✅ Sí |
| **Process** | Flujo de proceso (secuencial, jerárquico) | ❌ No (default: sequential) |
| **Verbose** | Nivel de verbosidad para logging | ❌ No (default: False) |
| **Manager LLM** | LLM usado por el agente manager en proceso jerárquico | ❌ No |
| **Function Calling LLM** | LLM para function calling de herramientas | ❌ No |
| **Config** | Configuración opcional del crew | ❌ No |
| **Max RPM** | Máximo requests por minuto | ❌ No |
| **Memory** | Memoria para almacenar ejecuciones | ❌ No |
| **Cache** | Cache para resultados de herramientas | ❌ No (default: True) |
| **Embedder** | Configuración del embedder | ❌ No |
| **Step Callback** | Función llamada después de cada paso | ❌ No |
| **Task Callback** | Función llamada después de cada tarea | ❌ No |
| **Share Crew** | Compartir información del crew | ❌ No |
| **Output Log File** | Archivo de logs de salida | ❌ No |
| **Manager Agent** | Agente manager personalizado | ❌ No |
| **Prompt File** | Archivo JSON de prompts | ❌ No |
| **Planning** | Capacidad de planificación | ❌ No |
| **Planning LLM** | LLM para planificación | ❌ No |
| **Knowledge Sources** | Fuentes de conocimiento disponibles | ❌ No |

## 1. Creación de Crews

Existen dos formas principales de crear crews en CrewAI:

### 1.1 Configuración YAML (Recomendado)

Usar configuración YAML proporciona una forma más limpia y mantenible de definir crews.

In [ ]:
# Ejemplo de Crew usando YAML Configuration
from crewai import Agent, Crew, Task, Process
from crewai.project import CrewBase, agent, task, crew, before_kickoff, after_kickoff
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List

@CrewBase
class ContentMarketingCrew:
    """Crew para crear contenido de marketing"""
    
    agents: List[BaseAgent]
    tasks: List[Task]
    
    # Rutas a archivos de configuración YAML
    agents_config = 'config/agents.yaml'
    tasks_config = 'config/tasks.yaml'
    
    @before_kickoff
    def prepare_inputs(self, inputs):
        # Modificar inputs antes de que el crew comience
        inputs['additional_data'] = "Información adicional del contexto"
        return inputs
    
    @after_kickoff
    def process_output(self, output):
        # Modificar output después de que el crew termine
        output.raw += "\nProcesado después del kickoff."
        return output
    
    @agent
    def researcher_agent(self) -> Agent:
        return Agent(
            config=self.agents_config['researcher'],
            verbose=True
        )
    
    @agent
    def writer_agent(self) -> Agent:
        return Agent(
            config=self.agents_config['writer'],
            verbose=True
        )
    
    @task
    def research_task(self) -> Task:
        return Task(
            config=self.tasks_config['research']
        )
    
    @task
    def write_task(self) -> Task:
        return Task(
            config=self.tasks_config['write']
        )
    
    @crew
    def crew(self) -> Crew:
        return Crew(
            agents=self.agents,  # Recolectado automáticamente por @agent
            tasks=self.tasks,    # Recolectado automáticamente por @task
            process=Process.sequential,
            verbose=True,
        )

# Cómo ejecutar el crew
# ContentMarketingCrew().crew().kickoff(inputs={"topic": "AI en marketing"})

### 1.2 Decoradores de CrewAI

CrewAI proporciona varios decoradores para marcar métodos dentro de tu clase crew:

- `@CrewBase`: Marca la clase como una clase base de crew
- `@agent`: Denota un método que retorna un objeto `Agent`
- `@task`: Denota un método que retorna un objeto `Task`
- `@crew`: Denota el método que retorna el objeto `Crew`
- `@before_kickoff`: (Opcional) Marca un método para ejecutarse antes de que el crew comience
- `@after_kickoff`: (Opcional) Marca un método para ejecutarse después de que el crew termine

### 1.3 Definición Directa en Código (Alternativa)

Alternativamente, puedes definir el crew directamente en código:

In [ ]:
# Ejemplo de Crew definido directamente en código
from crewai import Agent, Crew, Task, Process

class SimpleCrew:
    def __init__(self):
        # Definir agentes
        self.researcher = Agent(
            role='Investigador',
            goal='Investigar temas específicos',
            backstory='Eres un investigador experto en análisis de datos',
            verbose=True
        )
        
        self.writer = Agent(
            role='Escritor',
            goal='Escribir contenido basado en investigación',
            backstory='Eres un escritor creativo experto en comunicación',
            verbose=True
        )
        
        # Definir tareas
        self.research_task = Task(
            description='Investigar sobre el tema proporcionado',
            agent=self.researcher,
            expected_output='Informe de investigación detallado'
        )
        
        self.write_task = Task(
            description='Escribir un artículo basado en la investigación',
            agent=self.writer,
            expected_output='Artículo completo y bien estructurado',
            context=[self.research_task]
        )
        
        # Crear crew
        self.crew = Crew(
            agents=[self.researcher, self.writer],
            tasks=[self.research_task, self.write_task],
            process=Process.sequential,
            verbose=True
        )
    
    def run(self, topic):
        return self.crew.kickoff(inputs={"topic": topic})

# Uso
# simple_crew = SimpleCrew()
# result = simple_crew.run("Inteligencia Artificial en 2024")

## 2. Salida del Crew (Crew Output)

La salida de un crew en CrewAI está encapsulada dentro de la clase `CrewOutput`. Esta clase proporciona una forma estructurada de acceder a los resultados de la ejecución del crew.

### 2.1 Atributos de Crew Output

| Atributo | Tipo | Descripción |
|----------|------|-------------|
| **Raw** | `str` | La salida raw del crew (formato por defecto) |
| **Pydantic** | `Optional[BaseModel]` | Objeto Pydantic representando la salida estructurada |
| **JSON Dict** | `Optional[Dict[str, Any]]` | Diccionario representando la salida JSON |
| **Tasks Output** | `List[TaskOutput]` | Lista de objetos TaskOutput de cada tarea |
| **Token Usage** | `Dict[str, Any]` | Resumen del uso de tokens durante la ejecución |

In [ ]:
# Ejemplo de acceso a la salida del crew
import json

def analyze_crew_output(crew_output):
    """Analiza y muestra la salida del crew"""
    
    print("=== ANÁLISIS DE SALIDA DEL CREW ===\n")
    
    # Salida raw
    print(f"📄 Salida Raw:\n{crew_output.raw}\n")
    
    # Salida JSON
    if crew_output.json_dict:
        print(f"📊 Salida JSON:\n{json.dumps(crew_output.json_dict, indent=2, ensure_ascii=False)}\n")
    
    # Salida Pydantic
    if crew_output.pydantic:
        print(f"🔧 Salida Pydantic:\n{crew_output.pydantic}\n")
    
    # Salidas de tareas
    print(f"📋 Salidas de Tareas ({len(crew_output.tasks_output)} tareas):")
    for i, task_output in enumerate(crew_output.tasks_output, 1):
        print(f"  Tarea {i}: {task_output.raw[:100]}...")
    print()
    
    # Uso de tokens
    print(f"💾 Uso de Tokens:\n{json.dumps(crew_output.token_usage, indent=2, ensure_ascii=False)}\n")
    
    # Métodos adicionales
    print(f"📝 Representación String:\n{str(crew_output)}\n")
    
    if crew_output.json_dict:
        print(f"📋 Diccionario:\n{crew_output.to_dict()}")

# Ejemplo de uso
# result = my_crew.kickoff()
# analyze_crew_output(result)

## 3. Logs del Crew

Puedes ver logs en tiempo real de la ejecución del crew configurando `output_log_file`.

In [ ]:
# Ejemplos de configuración de logs

# 1. Guardar logs como logs.txt (por defecto)
crew_with_logs = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    output_log_file=True  # Guarda como logs.txt
)

# 2. Guardar logs con nombre personalizado
crew_with_custom_logs = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    output_log_file="mi_crew_log.txt"  # Guarda como mi_crew_log.txt
)

# 3. Guardar logs en formato JSON
crew_with_json_logs = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    output_log_file="mi_crew_log.json"  # Guarda como mi_crew_log.json
)

# 4. Sin logs
crew_without_logs = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    output_log_file=False  # No guarda logs
)

## 4. Procesos de Ejecución

CrewAI soporta diferentes tipos de procesos de ejecución:

### 4.1 Proceso Secuencial

Las tareas se ejecutan una después de otra, permitiendo un flujo lineal de trabajo.

In [ ]:
# Ejemplo de proceso secuencial
from crewai import Process

sequential_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, write_task, edit_task],
    process=Process.sequential,  # Tareas se ejecutan en orden
    verbose=True
)

# Flujo: research_task → write_task → edit_task

### 4.2 Proceso Jerárquico

Un agente manager coordina el crew, delegando tareas y validando resultados antes de proceder.

In [ ]:
# Ejemplo de proceso jerárquico
from crewai import Agent, Crew, Task, Process

# Agente manager
manager_agent = Agent(
    role='Project Manager',
    goal='Coordinar y supervisar el trabajo del equipo',
    backstory='Eres un manager experimentado en gestión de proyectos',
    verbose=True
)

# Crew jerárquico
hierarchical_crew = Crew(
    agents=[manager_agent, researcher, writer, editor],
    tasks=[research_task, write_task, edit_task],
    process=Process.hierarchical,
    manager_llm=llm,  # LLM para el manager (requerido)
    verbose=True
)

# Flujo: manager coordina → research_task → manager valida → write_task → manager valida → edit_task

## 5. Métodos de Kickoff

CrewAI proporciona varios métodos para iniciar la ejecución del crew:

### 5.1 Kickoff Básico

```python
result = my_crew.kickoff()
```

### 5.2 Kickoff para Múltiples Inputs

```python
inputs_array = [
    {'topic': 'AI en healthcare'},
    {'topic': 'AI en finance'},
    {'topic': 'AI en education'}
]
results = my_crew.kickoff_for_each(inputs=inputs_array)
```

### 5.3 Kickoff Asíncrono

```python
async_result = await my_crew.kickoff_async(inputs={'topic': 'AI en healthcare'})
```

### 5.4 Kickoff Asíncrono para Múltiples Inputs

```python
async_results = await my_crew.kickoff_for_each_async(inputs=inputs_array)
```

In [ ]:
# Ejemplo completo de diferentes métodos de kickoff
import asyncio

def demonstrate_kickoff_methods():
    """Demuestra los diferentes métodos de kickoff"""
    
    # 1. Kickoff básico
    print("🚀 Kickoff Básico:")
    # result = my_crew.kickoff()
    # print(f"Resultado: {result.raw[:100]}...")
    print("✅ Completado\n")
    
    # 2. Kickoff para múltiples inputs
    print("🔄 Kickoff para Múltiples Inputs:")
    inputs_array = [
        {'topic': 'AI en healthcare'},
        {'topic': 'AI en finance'},
        {'topic': 'AI en education'}
    ]
    # results = my_crew.kickoff_for_each(inputs=inputs_array)
    # for i, result in enumerate(results, 1):
    #     print(f"  Resultado {i}: {result.raw[:50]}...")
    print("✅ Completado\n")
    
    # 3. Kickoff asíncrono
    print("⚡ Kickoff Asíncrono:")
    # async_result = await my_crew.kickoff_async(inputs={'topic': 'AI en healthcare'})
    # print(f"Resultado asíncrono: {async_result.raw[:100]}...")
    print("✅ Completado\n")
    
    # 4. Kickoff asíncrono para múltiples inputs
    print("⚡🔄 Kickoff Asíncrono para Múltiples Inputs:")
    # async_results = await my_crew.kickoff_for_each_async(inputs=inputs_array)
    # for i, result in enumerate(async_results, 1):
    #     print(f"  Resultado asíncrono {i}: {result.raw[:50]}...")
    print("✅ Completado")

# Para ejecutar el ejemplo asíncrono
# asyncio.run(demonstrate_kickoff_methods())

## 6. Replay de Tareas

CrewAI permite hacer replay desde una tarea específica usando el CLI.

### 6.1 Ver Tareas Disponibles

```bash
crewai log-tasks-outputs
```

### 6.2 Replay desde una Tarea Específica

```bash
crewai replay -t <task_id>
```

### 6.3 Ejemplo de Uso

```bash
# Ver las últimas tareas ejecutadas
crewai log-tasks-outputs

# Replay desde la tarea con ID específico
crewai replay -t task_12345
```

## 7. Memoria y Cache

### 7.1 Memoria

Los crews pueden utilizar memoria para mejorar su ejecución y aprendizaje a lo largo del tiempo.

In [ ]:
# Ejemplo de crew con memoria
from crewai import Memory

crew_with_memory = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    memory=Memory(
        # Configuración de memoria
        short_term=True,
        long_term=True,
        entity_memory=True
    ),
    verbose=True
)

### 7.2 Cache

El cache puede emplearse para almacenar resultados de herramientas, haciendo el proceso más eficiente.

In [ ]:
# Ejemplo de crew con cache
crew_with_cache = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    cache=True,  # Habilita cache (por defecto)
    verbose=True
)

# O deshabilitar cache
crew_without_cache = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    cache=False,  # Deshabilita cache
    verbose=True
)

## 8. Métricas de Uso

Después de la ejecución del crew, puedes acceder a las métricas de uso del LLM.

In [ ]:
# Ejemplo de acceso a métricas de uso
def analyze_usage_metrics(crew):
    """Analiza las métricas de uso del crew"""
    
    # Ejecutar el crew
    result = crew.kickoff()
    
    # Acceder a métricas de uso
    usage_metrics = crew.usage_metrics
    
    print("📊 MÉTRICAS DE USO DEL CREW")
    print("=" * 50)
    
    for metric, value in usage_metrics.items():
        print(f"{metric}: {value}")
    
    print("=" * 50)
    
    return result

# Uso
# result = analyze_usage_metrics(my_crew)

## 9. Ejemplo Práctico Completo

Vamos a crear un ejemplo completo de un crew para crear contenido de marketing.

In [ ]:
# Ejemplo completo: Crew de Marketing de Contenido
from crewai import Agent, Crew, Task, Process
from langchain_openai import ChatOpenAI
import os

# Configurar LLM (asegúrate de tener tu API key configurada)
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7
)

class ContentMarketingCrew:
    def __init__(self):
        # 1. Agente Investigador
        self.researcher = Agent(
            role='Investigador de Mercado',
            goal='Investigar tendencias, datos y insights del mercado',
            backstory='''Eres un investigador experto en análisis de mercado. 
            Tienes años de experiencia en identificar tendencias, analizar datos 
            y proporcionar insights valiosos para estrategias de marketing.''',
            verbose=True,
            llm=llm
        )
        
        # 2. Agente Estratega
        self.strategist = Agent(
            role='Estratega de Marketing',
            goal='Desarrollar estrategias de marketing efectivas',
            backstory='''Eres un estratega de marketing creativo y analítico. 
            Especializado en convertir insights de investigación en estrategias 
            de marketing accionables y efectivas.''',
            verbose=True,
            llm=llm
        )
        
        # 3. Agente Creador de Contenido
        self.content_creator = Agent(
            role='Creador de Contenido',
            goal='Crear contenido de marketing atractivo y efectivo',
            backstory='''Eres un creador de contenido experto en marketing digital. 
            Especializado en escribir contenido que convierte, desde posts de redes 
            sociales hasta artículos de blog y copy publicitario.''',
            verbose=True,
            llm=llm
        )
        
        # 4. Agente Editor
        self.editor = Agent(
            role='Editor de Contenido',
            goal='Revisar y mejorar la calidad del contenido',
            backstory='''Eres un editor experimentado con un ojo agudo para la calidad. 
            Especializado en mejorar la claridad, el tono y la efectividad del contenido 
            de marketing.''',
            verbose=True,
            llm=llm
        )
        
        # Definir tareas
        self.setup_tasks()
        
        # Crear crew
        self.crew = Crew(
            agents=[self.researcher, self.strategist, self.content_creator, self.editor],
            tasks=[self.research_task, self.strategy_task, self.content_task, self.edit_task],
            process=Process.sequential,
            verbose=True,
            output_log_file=True  # Guardar logs
        )
    
    def setup_tasks(self):
        """Configurar las tareas del crew"""
        
        # Tarea 1: Investigación
        self.research_task = Task(
            description='''Investiga el tema proporcionado. Busca:
            - Tendencias actuales del mercado
            - Datos y estadísticas relevantes
            - Insights de la competencia
            - Oportunidades de mercado
            Proporciona un informe detallado con tus hallazgos.''',
            agent=self.researcher,
            expected_output='Informe de investigación detallado con datos y insights'
        )
        
        # Tarea 2: Estrategia
        self.strategy_task = Task(
            description='''Basándote en la investigación, desarrolla una estrategia de marketing:
            - Objetivos claros y medibles
            - Público objetivo definido
            - Canales de distribución
            - Mensajes clave
            - Métricas de éxito''',
            agent=self.strategist,
            expected_output='Estrategia de marketing completa y accionable',
            context=[self.research_task]
        )
        
        # Tarea 3: Creación de Contenido
        self.content_task = Task(
            description='''Crea contenido de marketing basado en la estrategia:
            - Artículo de blog principal
            - Posts para redes sociales
            - Copy publicitario
            - Email marketing
            Asegúrate de que el contenido sea atractivo y efectivo.''',
            agent=self.content_creator,
            expected_output='Contenido de marketing completo y variado',
            context=[self.research_task, self.strategy_task]
        )
        
        # Tarea 4: Edición
        self.edit_task = Task(
            description='''Revisa y mejora todo el contenido creado:
            - Corrige errores gramaticales y ortográficos
            - Mejora la claridad y el flujo
            - Optimiza para SEO
            - Asegura consistencia de tono y estilo
            - Verifica que cumple con la estrategia definida''',
            agent=self.editor,
            expected_output='Contenido final pulido y optimizado',
            context=[self.research_task, self.strategy_task, self.content_task]
        )
    
    def run(self, topic):
        """Ejecutar el crew con un tema específico"""
        print(f"🚀 Iniciando Content Marketing Crew para: {topic}")
        print("=" * 60)
        
        result = self.crew.kickoff(inputs={"topic": topic})
        
        print("\n✅ Crew completado exitosamente!")
        print("=" * 60)
        
        return result

# Ejemplo de uso
# marketing_crew = ContentMarketingCrew()
# result = marketing_crew.run("Inteligencia Artificial en el Marketing Digital 2024")

## 10. Ejercicios Prácticos

### Ejercicio 1: Crew Básico

Crea un crew simple con dos agentes: un investigador y un escritor. El investigador debe buscar información sobre un tema y el escritor debe crear un artículo basado en esa investigación.

### Ejercicio 2: Crew con Proceso Jerárquico

Crea un crew con proceso jerárquico que incluya:
- Un agente manager
- Un agente investigador
- Un agente escritor
- Un agente editor

### Ejercicio 3: Crew con Múltiples Inputs

Crea un crew que procese múltiples temas simultáneamente usando `kickoff_for_each()`.

### Ejercicio 4: Crew con Logs y Métricas

Crea un crew que guarde logs detallados y analice las métricas de uso después de la ejecución.

### Ejercicio 5: Crew Personalizado

Crea un crew para tu propio caso de uso específico (por ejemplo, análisis de datos, creación de código, planificación de eventos, etc.).

## 11. Mejores Prácticas

### 11.1 Diseño de Crews

1. **Define roles claros**: Cada agente debe tener un propósito específico
2. **Establece dependencias**: Las tareas deben tener un flujo lógico
3. **Usa contexto**: Pasa información entre tareas cuando sea necesario
4. **Configura logging**: Siempre habilita logs para debugging

### 11.2 Optimización

1. **Usa cache**: Habilita cache para tareas repetitivas
2. **Configura RPM**: Establece límites de requests por minuto
3. **Monitorea métricas**: Revisa el uso de tokens regularmente
4. **Prueba diferentes procesos**: Experimenta con secuencial vs jerárquico

### 11.3 Debugging

1. **Revisa logs**: Usa `output_log_file=True`
2. **Analiza métricas**: Monitorea `usage_metrics`
3. **Usa replay**: Utiliza el CLI para replay de tareas específicas
4. **Verifica dependencias**: Asegúrate de que las tareas tengan el contexto correcto

## 12. Resumen

### Conceptos Clave

- **Crew**: Grupo colaborativo de agentes trabajando juntos
- **Process**: Flujo de ejecución (secuencial o jerárquico)
- **Tasks**: Tareas asignadas al crew
- **Agents**: Agentes que forman parte del crew
- **Kickoff**: Método para iniciar la ejecución
- **CrewOutput**: Estructura de salida del crew

### Métodos de Kickoff

- `kickoff()`: Ejecución básica
- `kickoff_for_each()`: Múltiples inputs secuencialmente
- `kickoff_async()`: Ejecución asíncrona
- `kickoff_for_each_async()`: Múltiples inputs asíncronamente

### Características Avanzadas

- **Memoria**: Almacenamiento de ejecuciones previas
- **Cache**: Almacenamiento de resultados de herramientas
- **Logs**: Registro detallado de ejecución
- **Métricas**: Análisis de uso de tokens
- **Replay**: Repetición desde tareas específicas

### Próximos Pasos

1. Experimenta con diferentes tipos de procesos
2. Crea crews para casos de uso específicos
3. Optimiza el rendimiento con cache y métricas
4. Implementa logging y debugging
5. Explora características avanzadas como memoria y planificación

## Referencias

- [Documentación oficial de Crews](https://docs.crewai.com/en/concepts/crews.md)
- [CrewAI GitHub](https://github.com/joaomdmoura/crewAI)
- [Ejemplos de CrewAI](https://github.com/joaomdmoura/crewAI/tree/main/examples)

---

*Este tutorial fue creado basado en la documentación oficial de CrewAI. Para más información, consulta la documentación completa en https://docs.crewai.com/*